In [1]:
import numpy as np
import osmnx as ox
import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt

import random
import pickle
import pymde
from sklearn.manifold import MDS, Isomap, TSNE, LocallyLinearEmbedding, SpectralEmbedding

import mlrfit as mf
import lrrouting as ldr
from lrrouting import convex_concave, convex_concave_forloop, compute_lq_norm_matrix, norm_all_subgradients, norm_subgradient, obj_distortion_function

import cvxpy as cp

/Users/parshakova.tanya/opt/anaconda3/envs/routing/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
np.random.seed(1001)
random.seed(1001)

#  Matrix definition

In [3]:
rank = 20

mtype = "small_world"
n = 200

G = nx.connected_watts_strogatz_graph(n, k=10, p=0.1)
G.remove_edges_from(nx.selfloop_edges(G))

n = G.number_of_nodes()
print(f"{n=}, {G.number_of_edges()=}")

for u, v in G.edges():
    G[u][v]['weight'] = np.random.rand() * 10

Adj, Dist = ldr.nx_graph_to_matrices(G)
A = Dist

n=200, G.number_of_edges()=1000
in  degrees: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: 0, 8: 8, 9: 44, 10: 98, 11: 35, 12: 13, 13: 0, 14: 1}
out degrees: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: 0, 8: 8, 9: 44, 10: 98, 11: 35, 12: 13, 13: 0, 14: 1}


In [4]:
assert nx.is_connected(G)
np.histogram(Dist.flatten(), bins=5, density=True)

(array([0.02066637, 0.08433458, 0.14056775, 0.05517102, 0.00273124]),
 array([ 0.        ,  3.29520811,  6.59041622,  9.88562432, 13.18083243,
        16.47604054]))

In [5]:
adjacency_list = ldr.adjacency_list(Adj)
sources, targets = ldr.st_pairs(n, Dist, 1020)
M = min(1000, sources.size)
sources = sources[:M]
targets = targets[:M]

In [6]:
PSD = False
w_min = A[A>0].min()
rt_max_iters = min(int(5*A.max()/w_min), (10**4) // 2)
symm = np.allclose(A, A.T)
print(f"{symm=}")
filename = "%s_r%d_%d"%(mtype, rank, n)

symm=True


In [7]:
np.histogram(Adj[Adj>0], bins=5, density=True)

(array([0.10560921, 0.10661024, 0.09359679, 0.1026061 , 0.09209523]),
 array([9.81857711e-03, 2.00775041e+00, 4.00568224e+00, 6.00361407e+00,
        8.00154590e+00, 9.99947774e+00]))

In [8]:
info = {} 

# $\ell_\infty$-norm and $\ell_2$-norm based embeddings

In [9]:
perm = np.random.permutation(n)
for ord in [2, np.inf]:
    norm_ord = lambda a: norm_subgradient(a, ord=ord)
    X_emb = np.random.randn(n, 20)
    for t in range(n):
        i = perm[t]
        g_i = np.apply_along_axis(norm_ord, 1, np.repeat(X_emb[i:i+1], n, axis=0) - X_emb)
        x_i1, obj_i1 = convex_concave(Dist[i], i, X_emb, ord, solver=cp.CLARABEL, g_i=g_i)
        x_i2, obj_i2 = convex_concave_forloop(Dist[i], i, X_emb, ord, solver=cp.CLARABEL, g_i=g_i)
        assert mf.rel_diff(obj_i1, obj_i2) < 1e-6, print(t, obj_i1, obj_i2, mf.rel_diff(obj_i1, obj_i2))
        assert mf.rel_diff(x_i1, x_i2) < 1e-3, print(t, mf.rel_diff(x_i1, x_i2))
        if ord == 2:
            x_i3, obj_i3 = ldr.bcd_l2_single(n, rank, i, Dist[i], X_emb, g_i=g_i)
            assert mf.rel_diff(obj_i1, obj_i3) < 1e-6, print(t, obj_i1, obj_i3, mf.rel_diff(obj_i1, obj_i3))
            assert mf.rel_diff(x_i1, x_i3) < 1e-3, print(t, mf.rel_diff(x_i1, x_i3))
        X_emb[i] = x_i1
print("PASSED")

/Users/parshakova.tanya/opt/anaconda3/envs/routing/lib/python3.9/site-packages/cvxpy/problems/problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


PASSED


In [10]:
X_emb = np.random.randn(n, 20)
for ord in [2, np.inf]: 
    for i in range(n):
        norm_ord = lambda a: norm_subgradient(a, ord=ord)
        g_matrix = np.apply_along_axis(norm_ord, 1, np.repeat(X_emb[i:i+1], n, axis=0) - X_emb)
        for j in range(n):
            if i == j: continue
            g_ij = norm_subgradient(X_emb[i] - X_emb[j], ord=ord)
            assert np.allclose(g_ij, g_matrix[j])

print("PASSED")

PASSED


In [11]:
for _ in range(100):
    for ord in [2, np.inf]:
        a = np.random.randn(n)
        g = norm_subgradient(a, ord=ord)
        assert np.allclose(np.linalg.norm(g, ord=1/(1-1/ord)), 1)
        assert np.allclose(np.linalg.norm(a, ord=ord), g.T @ (a)), print(np.linalg.norm(a, ord=ord), g.T @ (a))
print("PASSED")


def test_lp_norm_matrix(X, ord, M1):
    n = X.shape[0]
    M2 = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            lp_norm = np.linalg.norm(X[i] - X[j], ord=ord)
            M2[i, j] = lp_norm
    return np.allclose(M2, M1)


for _ in range(10):
    for ord in [2, 3, np.inf]:
        x1 = np.random.randn(n, n//2)
        d2 = compute_lq_norm_matrix(x1, ord)
        test_lp_norm_matrix(x1, ord, d2)
print("PASSED")

PASSED
PASSED


In [12]:
# verify that l-infinity is a universal embedding
D_linf = compute_lq_norm_matrix(Dist[:, :-1], np.inf)
assert np.allclose(np.linalg.norm(D_linf - Dist, 'fro'), 0)

In [13]:
fraction_of_nodes = 0.1
pi = np.random.permutation(n)[:int(n * fraction_of_nodes)]

In [14]:
# check if the method is a descent method
max_iter = 200

for ord in [np.inf, 2]:
    norm_ord = lambda a: norm_subgradient(a, ord=ord)
    X_emb = np.random.randn(n, rank)
    losses = [obj_distortion_function(Dist, X_emb, ord)]
    print(f"{ord=}, distortion={losses[-1]}")
    for t in range(max_iter):
        for i in pi:
            g_i = np.apply_along_axis(norm_ord, 1, np.repeat(X_emb[i:i+1], n, axis=0) - X_emb)
            x_i, obj_i = convex_concave(Dist[i], i, X_emb, ord, solver=cp.CLARABEL, g_i=g_i)
            losses += [obj_distortion_function(Dist, X_emb, ord)]
            X_emb[i] = x_i
        if t%50 == 0 or t == max_iter-1:
            print(f"{t=}, distortion={losses[-1]}")

    assert (np.diff(losses) < 1e-6).all(), print("it is not a descent method")
    assert X_emb.shape == (n, rank)
    linf_dar = ldr.construct_node_embedding_graph(X_emb, adjacency_list)
    info["nemb_l%s"%str(ord)] = {'ratios' : ldr.subopt_ratios(linf_dar, Dist, sources, targets)}

ord=inf, distortion=55.019701911156446
t=0, distortion=47.72758444075375
t=50, distortion=47.15410038911772
t=100, distortion=47.068484584132115
t=150, distortion=47.004007800806804
t=199, distortion=46.966952127337144
median_stretch=739.9%, mean_stretch=1095.4%
%[ratio<2] = 14.50%, %[ratio<1.2] = 6.60%, %[ratio=1.] = 4.50%
ord=2, distortion=19.718254457796636
t=0, distortion=18.838483924579446
t=50, distortion=18.22807611879186
t=100, distortion=18.099596138795857
t=150, distortion=18.06557923791513
t=199, distortion=18.05295478745643
median_stretch=602.7%, mean_stretch=910.1%
%[ratio<2] = 16.90%, %[ratio<1.2] = 7.80%, %[ratio=1.] = 4.70%


# Classical MDS

In [15]:
X_cmds, G_cmds = ldr.classical_mds((Dist + Dist.T)/2, rank)
assert X_cmds.shape == (n, rank)
mds_dar = ldr.construct_node_embedding_graph(X_cmds, adjacency_list)
info['cmds'] = {'ratios' : ldr.subopt_ratios(mds_dar, Dist, sources, targets)}

median_stretch=104.8%, mean_stretch=133.3%
%[ratio<2] = 90.10%, %[ratio<1.2] = 64.80%, %[ratio=1.] = 40.80%


# MDS

In [16]:
mds = MDS(n_components=rank, metric=True, n_init=10, verbose=True, max_iter=100, dissimilarity='precomputed') 
mds_embedding = mds.fit_transform((Dist + Dist.T)/2)
assert mds_embedding.shape == (n, rank)
mds_dar = ldr.construct_node_embedding_graph(mds_embedding, adjacency_list)
info['mds'] = {'ratios' : ldr.subopt_ratios(mds_dar, Dist, sources, targets)}

median_stretch=103.8%, mean_stretch=134.0%
%[ratio<2] = 90.80%, %[ratio<1.2] = 66.70%, %[ratio=1.] = 43.20%


# Routing

In [17]:
frac = 1.2
for method in info.keys():
    rats = 100.*(info[method]['ratios'] <= frac + 1e-8).sum() / info[method]['ratios'].size
    print(f"{method:12s}   %[ratio<{frac}]={rats:.2f}%")

nemb_linf      %[ratio<1.2]=6.60%
nemb_l2        %[ratio<1.2]=7.80%
cmds           %[ratio<1.2]=64.80%
mds            %[ratio<1.2]=66.70%


In [18]:
# ldr.plot_cdf_algo_subopt_ratio(info, title=f"{mtype}, {rank=}")

In [19]:
_ = ldr.subopt_ratios_mtrx(Dist, adjacency_list, Adj, Dist, sources, targets, rt_max_iters)

median_stretch=100.0%, mean_stretch=100.0%
%[ratio<2] = 100.00%, %[ratio<1.2] = 100.00%, %[ratio=1.] = 100.00%


In [20]:
for (s, t) in zip(sources, targets):
    route_lr, w_lr = mds_dar.route(s, t)
    assert ldr.valid_path(route_lr, w_lr, adjacency_list, s, t)
print("PASSED")

PASSED


In [21]:
s = 50; t = 20
route_lr, w_lr = mds_dar.route(s, t)
assert ldr.valid_path(route_lr, w_lr, adjacency_list, s, t)